In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LinearRegression
%matplotlib inline
from statsmodels.formula.api import ols
import numpy as np
import src
import random
import scipy.stats as stats



# Methodology

My approach to this model was to do some eda to find strongly correlated variables, then work from the ground up to add features one at a time and see their effect. As such this notebook can dawdle somewhat, but I did not want to deviate its structure too much from my though process. 

In [67]:
df = pd.read_csv("kc_house_data_test_features.csv")
fedfunds = pd.read_csv("fedfunds")
fedfunds['year_month'] = fedfunds['year_month'].astype(str)
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [68]:
df.grade.value_counts()

8     1464
7     1439
9      716
10     294
6      256
11     100
5       33
12      14
4        5
13       1
1        1
Name: grade, dtype: int64

This visualization is a deeper dive on location where the average prices of each zip houses sold in each zipcode. Price seems to vary drastically by zipcode. This is expected since zipcode is our proxy for urbanization, school districts etc... 98038 has the highest zipcode, 98002 seems to be the lowest. It will be interesting to see which zipcodes are significant in our model.

In [69]:
df = pd.get_dummies(df, columns = ['zipcode'], drop_first = True)


In [70]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
df['sq_sqft'] = df['sqft_living'] **2


In [72]:
df['yrs_old'] = 2020 - df.yr_built

In [73]:
df['has_been_renovated'] = df['yr_renovated'].apply(lambda x: x>0).astype(int)

In [74]:
df = pd.get_dummies(df, columns = ['grade'], drop_first = True)

In [87]:
df['grade_3'] = 0

In [88]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade_10,grade_11,grade_12,grade_13,month,fall,winter,spring,summer,grade_3
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,8,0,0,0,1,0
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,2,0,1,0,0,0
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,0,0,0,0,11,1,0,0,0,0
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,0,0,0,0,12,1,0,0,0,0
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [89]:
df['month'] = df['date'].apply(lambda x: x[4:6])

In [90]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade_10,grade_11,grade_12,grade_13,month,fall,winter,spring,summer,grade_3
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,08,0,0,0,1,0
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,02,0,1,0,0,0
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,0,0,0,0,11,1,0,0,0,0
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,0,0,0,0,12,1,0,0,0,0
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,0,0,0,0,01,0,1,0,0,0


In [91]:
df['month'] = df['month'].astype(int)
df['fall'] = df['month'].apply(lambda x: x>9).astype(int)
df['winter'] = df['month'].apply(lambda x: x<3).astype(int)
df['spring'] = df['month'].apply(lambda x: 6>x>3).astype(int)
df['summer'] = df['month'].apply(lambda x: 9>x>6).astype(int)
df.head(20)

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade_10,grade_11,grade_12,grade_13,month,fall,winter,spring,summer,grade_3
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,8,0,0,0,1,0
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,2,0,1,0,0,0
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,0,0,0,0,11,1,0,0,0,0
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,0,0,0,0,12,1,0,0,0,0
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,0,0,0,0,1,0,1,0,0,0
5,5,1870400605,20141117T000000,4,2.25,1970,7125,1.5,0,0,...,0,0,0,0,11,1,0,0,0,0
6,6,4047200825,20141011T000000,1,1.00,1390,60984,1.0,0,0,...,0,0,0,0,10,1,0,0,0,0
7,7,7304300430,20141201T000000,4,1.00,1020,11383,1.5,0,0,...,0,0,0,0,12,1,0,0,0,0
8,8,8079010190,20140626T000000,4,2.50,2250,7526,2.0,0,0,...,0,0,0,0,6,0,0,0,0,0
9,9,1453600182,20140528T000000,2,1.00,800,6240,1.0,0,0,...,0,0,0,0,5,0,0,1,0,0


Not much of an effect, also, only winter and fall are significant.

In [102]:
import pickle
model = open("model.pickle", "rb")
pickle.load(model)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [103]:
from sklearn import linear_model

In [104]:
final_model = pickle.load(open('model.pickle','rb'))

In [105]:
factors = """zipcode_98004+zipcode_98005 +
                            zipcode_98006+zipcode_98007+zipcode_98008+zipcode_98010 
                            +zipcode_98011+zipcode_98014+zipcode_98019 
                            +zipcode_98024+zipcode_98027+zipcode_98028 
                            +zipcode_98029+
                            +zipcode_98033+zipcode_98034+zipcode_98039 
                            +zipcode_98040+zipcode_98045+zipcode_98052 
                            +zipcode_98053+zipcode_98056
                            +zipcode_98059+zipcode_98065+zipcode_98072 
                            +zipcode_98074+zipcode_98075+zipcode_98077+zipcode_98092 
                            +zipcode_98102+zipcode_98103+zipcode_98105 
                            +zipcode_98107+zipcode_98108+zipcode_98109+zipcode_98112 
                            +zipcode_98115+zipcode_98116+zipcode_98117+zipcode_98118 
                            +zipcode_98119+zipcode_98122+zipcode_98125+zipcode_98126 
                            +zipcode_98133+zipcode_98136+zipcode_98144+zipcode_98146 
                            +zipcode_98155+zipcode_98166 
                            +zipcode_98177+zipcode_98199+sq_sqft
                            +grade_3+sqft_living
                            +grade_4+grade_5+grade_6+grade_7+grade_8+bedrooms+bathrooms+floors+grade_9+grade_10+grade_11+grade_12+waterfront+winter+fall"""
factors = factors.replace(' ', '')
factors = factors.replace('\n', '')
factors = factors.split('+')
factors.remove('' )
df[factors]

,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98024,...,bedrooms,bathrooms,floors,grade_9,grade_10,grade_11,grade_12,waterfront,winter,fall
0,0,0,0,0,0,0,0,0,0,0,...,4,2.50,1.0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,4,2.50,1.0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,3,2.50,2.0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,3,1.75,1.0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,4,2.75,2.0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,0,0,0,0,0,0,0,0,0,0,...,3,2.50,3.0,0,0,0,0,0,0,0
4319,0,0,0,0,0,0,0,0,0,0,...,4,2.50,2.0,0,0,0,0,0,1,0
4320,0,0,0,0,0,0,0,0,0,0,...,2,0.75,2.0,0,0,0,0,0,0,0
4321,0,0,0,0,0,0,0,0,0,0,...,3,2.50,2.0,0,0,0,0,0,1,0


In [106]:
final_model.predict(df[factors])

array([564052.97262196, 556348.28752588, 438307.40342464, ...,
       430219.17728084, 409367.70191727, 415695.22430269])